In [ ]:
# Install Dependencies

!pip install -q sentence-transformers pymongo python-dotenv tqdm

In [48]:
# Import Dependencies

from sentence_transformers import SentenceTransformer
from pymongo import MongoClient
from dotenv import load_dotenv
from tqdm import tqdm
import os

In [49]:
# Load the environment variables

load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
DB_NAME = os.getenv("DATABASE_NAME")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")

if not all([MONGO_URI, DB_NAME, COLLECTION_NAME]):
    raise ValueError("Missing environment variables. Check your .env file")

In [ ]:
# Connect to MongoDB

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

print(f"Connected to MongoDB!")

In [ ]:
# Load embedding model

model = SentenceTransformer("all-MiniLM-L6-v2")
print("Embedding model loaded successfully")

In [ ]:
# Load a sample dataset

dataset = [
    "AI@UCF is the only AI club at UCF",
    "AI@UCF has 4 sections: GBMs, Projects, CORE (workshops), and Discussions (discuss research papers)",
    "AI@UCF has 3 different projects variants: Research, Sponsorship, and General",
    "Lucia is the President and GBM Director",
    "Gaurav is the VP and Project Director",
    "Hema is the Secretary and CORE Director",
    "Geno is the Discussion Director",
    "Anthony is the Tresuerer",
    "There are 3 different types of memebers: members, coordinators, and E-board & Directors",
    "Coordinators help plan events and general logistics of the club",
    "Coordinator applications are open",
    "There is no membership fees to"
]

print(f"Loaded {len(dataset)} documents")

In [ ]:
# Embed the dataset and store into the vectorDB

for text in tqdm(dataset, desc="Uploading to MongoDB"):
    embedding = model.encode(text, normalize_embeddings=True).tolist()

    record = {
        "text": text,
        "embedding": embedding
    }

    collection.update_one({"text": text}, {"$set": record}, upsert=True)
    
print("All documents embedded and uploaded successfully!")

In [ ]:
# Get Vector Embedding
query_text = "Artificial intelligence"
query_embedding = model.encode(query_text).tolist()

# Run vector search
pipeline = [
    {
        "$vectorSearch": {
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 100,
            "limit": 3,
            "index": "vector_index",  # use your actual index name if different
            "similarity": "cosine"
        }
    },
    {
        "$project": {
            "_id": 0,
            "text": 1,
            "score": {"$meta": "vectorSearchScore"}
        }
    }
]

results = list(collection.aggregate(pipeline))

# Print the top matches
print("🔍 Top Retrieved Results:")
for r in results:
    print(f"Score: {r['score']:.4f} | Text: {r['text'][:150]}...")